In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [4]:
# imports
import pandas as pd
import pandas_datareader.data as pdr
import datetime
import talib
from talib.abstract import *
from talib import MA_Type

# format price data
pd.options.display.float_format = '{:0.2f}'.format

In [6]:
symbol = 'SPY'
start = datetime.datetime(2018, 1, 1)
end = datetime.datetime.now()

In [7]:
ts = pdr.DataReader(symbol, 'yahoo', start, end)
ts.tail()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2022-02-16,448.06,441.94,443.93,446.60,84863600.00,446.60
2022-02-17,446.57,436.42,443.22,437.06,102259100.00,437.06
2022-02-18,438.66,431.82,437.33,434.23,132454300.00,434.23
2022-02-22,435.50,425.86,431.89,429.57,124391800.00,429.57
2022-02-23,433.26,421.35,432.66,421.95,132000000.00,421.95


In [8]:
def _adj_column_names(ts):
    """
    ta-lib expects columns to be lower case; to be consistent,
    change date index
    """
    ts.columns = [col.lower().replace(' ','_') for col in ts.columns]
    ts.index.names = ['date']
    return ts

ts = _adj_column_names(ts)

In [9]:
print('There are {} TA-Lib functions!'.format(len(talib.get_functions())))

There are 158 TA-Lib functions!


In [11]:
for group, funcs in talib.get_function_groups().items():
    print(group)
    print('-----------------------------------------')
    for func in funcs:
        f = Function(func)
        print('{} - {}'.format(func, f.info['display_name']))
    print()

Cycle Indicators
-----------------------------------------
HT_DCPERIOD - Hilbert Transform - Dominant Cycle Period
HT_DCPHASE - Hilbert Transform - Dominant Cycle Phase
HT_PHASOR - Hilbert Transform - Phasor Components
HT_SINE - Hilbert Transform - SineWave
HT_TRENDMODE - Hilbert Transform - Trend vs Cycle Mode

Math Operators
-----------------------------------------
ADD - Vector Arithmetic Add
DIV - Vector Arithmetic Div
MAX - Highest value over a specified period
MAXINDEX - Index of highest value over a specified period
MIN - Lowest value over a specified period
MININDEX - Index of lowest value over a specified period
MINMAX - Lowest and highest values over a specified period
MINMAXINDEX - Indexes of lowest and highest values over a specified period
MULT - Vector Arithmetic Mult
SUB - Vector Arithmetic Substraction
SUM - Summation

Math Transform
-----------------------------------------
ACOS - Vector Trigonometric ACos
ASIN - Vector Trigonometric ASin
ATAN - Vector Trigonometric AT

In [12]:
print(SMA)

SMA([input_arrays], [timeperiod=30])

Simple Moving Average (Overlap Studies)

Inputs:
    price: (any ndarray)
Parameters:
    timeperiod: 30
Outputs:
    real


In [13]:
print(SMA.info)

{'name': 'SMA', 'group': 'Overlap Studies', 'display_name': 'Simple Moving Average', 'function_flags': ['Output scale same as input'], 'input_names': OrderedDict([('price', 'close')]), 'parameters': OrderedDict([('timeperiod', 30)]), 'output_flags': OrderedDict([('real', ['Line'])]), 'output_names': ['real']}


In [14]:
sma = SMA(ts)
sma.tail()

date
2022-02-16   451.48
2022-02-17   450.44
2022-02-18   449.32
2022-02-22   448.10
2022-02-23   446.65
dtype: float64

In [15]:
sma200 = SMA(ts, timeperiod=200)
sma200.tail()

date
2022-02-16   444.52
2022-02-17   444.62
2022-02-18   444.70
2022-02-22   444.74
2022-02-23   444.76
dtype: float64

In [16]:
sma200 = SMA(ts, timeperiod=200, price='open')
sma200.tail()

date
2022-02-16   444.50
2022-02-17   444.63
2022-02-18   444.74
2022-02-22   444.80
2022-02-23   444.85
dtype: float64

In [17]:
ts['sma200'] = sma200
ts.tail()

,high,low,open,close,volume,adj_close,sma200
date,,,,,,,
2022-02-16,448.06,441.94,443.93,446.60,84863600.00,446.60,444.50
2022-02-17,446.57,436.42,443.22,437.06,102259100.00,437.06,444.63
2022-02-18,438.66,431.82,437.33,434.23,132454300.00,434.23,444.74
2022-02-22,435.50,425.86,431.89,429.57,124391800.00,429.57,444.80
2022-02-23,433.26,421.35,432.66,421.95,132000000.00,421.95,444.85


In [18]:
#upper, middle, lower = BBANDS(ts, timeperiod=20, matype=MA_Type.EMA)
#(for some reason, the abstract API doesn't work for BBANDS, so use the function API)

upper, middle, lower = talib.BBANDS(ts.close, timeperiod=20, matype=MA_Type.EMA)
ts['upper'] = upper; ts['middle'] = middle; ts['lower'] = lower
ts.tail()

,high,low,open,close,volume,adj_close,sma200,upper,middle,lower
date,,,,,,,,,,
2022-02-16,448.06,441.94,443.93,446.60,84863600.00,446.60,444.50,463.27,448.73,434.20
2022-02-17,446.57,436.42,443.22,437.06,102259100.00,437.06,444.63,462.52,447.62,432.73
2022-02-18,438.66,431.82,437.33,434.23,132454300.00,434.23,444.74,461.65,446.35,431.04
2022-02-22,435.50,425.86,431.89,429.57,124391800.00,429.57,444.80,461.25,444.75,428.24
2022-02-23,433.26,421.35,432.66,421.95,132000000.00,421.95,444.85,461.25,442.58,423.91
